In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

from avalanche.benchmarks.classic import CLStream51
#from avalanche.models import MobilenetV1
from pretrained_model.resnet18_places365 import build_resnet18
from avalanche.training import CWRStar
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.logging.text_logging import TextLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    forgetting_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    ram_usage_metrics,
    timing_metrics,
)



c:\Users\USER\anaconda3\envs\avalanche-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Config
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Eval_metics
wandb_logger = WandBLogger(run_name="CWR*_stream51")
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    gpu_usage_metrics(gpu_id=0,experience=True),
    ram_usage_metrics(experience=True),
    timing_metrics(minibatch=True, epoch=True, experience=True),
    loggers=[InteractiveLogger(), wandb_logger,
                TextLogger(open('cwr_star_results.txt', 'w'))]
)

# model
model = build_resnet18()

# Prepare for training & testing
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

# CL Benchmark Creation
benchmark = CLStream51(scenario="class_instance", bbox_crop=True) 
# scenario
# "iid", "class_iid", "instance", "class_instance"

train_stream = benchmark.train_stream
test_stream = benchmark.test_stream

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jgpark (cais). Use `wandb login --relogin` to force relogin


E:\python\CL\avalanche_root\avalanche\training\plugins\evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


Shape mismatch...ignoring fc.weight
Shape mismatch...ignoring fc.bias
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Continual learning strategy
from avalanche.training.utils import replace_bn_with_brn
replace_bn_with_brn(
            model,
            momentum=0.01,
            r_d_max_inc_step=4.1e-05,
            max_r_max=1.25,
            max_d_max=0.5,
        )

cl_strategy = CWRStar(
    model,
    optimizer,
    criterion,
    cwr_layer_name=None,
    train_mb_size=128, train_epochs=4,
    eval_mb_size=128, device=device,
    evaluator=eval_plugin
)

# train and test loop over the stream of experiences
results = []
for train_exp in train_stream:
    cl_strategy.train(train_exp)
    results.append(cl_strategy.eval(test_stream))

-- >> Start of training phase << --
100%|██████████| 238/238 [06:47<00:00,  1.71s/it]
Epoch 0 ended.
	Time_Epoch/train_phase/train_stream/Task000 = 407.2324
	Time_MB/train_phase/train_stream/Task000 = 0.1810
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8248
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9753
100%|██████████| 238/238 [06:44<00:00,  1.70s/it]
Epoch 1 ended.
	Time_Epoch/train_phase/train_stream/Task000 = 404.4929
	Time_MB/train_phase/train_stream/Task000 = 0.1621
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9826
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9877
100%|██████████| 238/238 [06:46<00:00,  1.71s/it]
Epoch 2 ended.
	Time_Epoch/train_phase/train_stream/Task000 = 406.3040
	Time_MB/train_phase/train_stream/Task000 = 0.1680
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9949
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 238/238 [06:45<00:00,  1.70s/it]
Epoch 3 ended.
	Time_Epoch/train_phase/train_stream/Task000